In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = './train_log/transfer/test/'
logger.set_logger_dir(log_dir)

[1029 11:03:46 @logger.py:94] WRN Log directory ./train_log/transfer/test/ exists! Please either backup/delete it, or use a new directory.
[1029 11:03:46 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1029 11:03:46 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1029 11:03:48 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-453e813e-e305-4457-bca8-d100773d6d44.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

#### Split train set and test set

In [7]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

Group numbers:
train: 5014, validation: 0, test: 4726
Image numbers:
train: 22989, validation: 0, test: 21028



#### Gether data to train rnn

In [10]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.annotation_number = 30
dm = DataManager.from_dataset(train_set, test_set, config)

Group numbers:
train: 3583, validation: 829, test: 0
Image numbers:
train: 16164, validation: 3651, test: 0

Group numbers:
train: 0, validation: 0, test: 4049
Image numbers:
train: 0, validation: 0, test: 17464



In [8]:
dm.get_imbalance_ratio()

/home/fuxiaofeng/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:167: RuntimeWarning: invalid value encountered in double_scalars
  posi_ratio = np.sum(binary_annot, axis=0) / binary_annot.shape[0]
/home/fuxiaofeng/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:168: RuntimeWarning: divide by zero encountered in true_divide
  return (1 - posi_ratio) / posi_ratio


,train,val,test
cellular blastoderm,10.881517,NaN,11.436842
maternal,28.151163,NaN,30.092105
ubiquitous,1.935597,NaN,2.154873
trunk mesoderm primordium,7.765734,NaN,8.433134
head mesoderm primordium P2,18.509728,NaN,20.098214
anterior endoderm primordium,19.299595,NaN,20.778802
posterior endoderm primordium,18.064639,NaN,19.911504
anterior midgut primordium,9.782796,NaN,10.583333
posterior midgut primordium,9.380952,NaN,9.990698
hindgut proper primordium,18.210728,NaN,19.370690


In [5]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = False

threshold = 0.5
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config, is_finetuning=False)

KeyError: 'embryonic central brain glia'

In [6]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()

[1028 20:56:05 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 12
[1028 20:56:06 @input_source.py:178] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1028 20:56:06 @input_source.py:459] Setting up StagingArea for GPU prefetching ...
[1028 20:56:06 @training.py:41] Training a model of 2 towers
[1028 20:56:06 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
IN

[1028 20:56:21 @base.py:212] Creating the session ...
[1028 20:56:24 @base.py:216] Initializing the session ...
[1028 20:56:24 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1028 20:56:25 @base.py:223] Graph Finalized.
[1028 20:56:25 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1028 20:56:25 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1028 20:56:27 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1028 20:56:27 @input_source.py:418] Pre-filling staging area ...
[1028 20:56:31 @base.py:257] Start Epoch 1 ...


100%|##########|55/55[02:03<00:00, 0.40it/s]

[1028 20:58:34 @base.py:267] Epoch 1 (global_step 55) finished, time:123.55 sec.



  0%|          |0/12[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|12/12[00:10<00:00, 1.21it/s]


...

[1028 21:36:47 @saver.py:89] Model saved to train_log/transfer/all_stages-g2/model-1045.
[1028 21:36:48 @saver.py:158] Model with maximum 'micro_auc' saved.
[1028 21:36:48 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.734
[1028 21:36:48 @monitor.py:361] QueueInput/queue_size_1: 49.694
[1028 21:36:48 @monitor.py:361] coverage: 4.0703
[1028 21:36:48 @monitor.py:361] learning_rate: 1e-05
[1028 21:36:48 @monitor.py:361] loss/value: 0.034225
[1028 21:36:48 @monitor.py:361] macro_auc: 0.93739
[1028 21:36:48 @monitor.py:361] macro_f1: 0.43674
[1028 21:36:48 @monitor.py:361] mean_average_precision: 0.67921
[1028 21:36:48 @monitor.py:361] micro_auc: 0.9483
[1028 21:36:48 @monitor.py:361] micro_f1: 0.5522
[1028 21:36:48 @monitor.py:361] one_error: 0.30729
[1028 21:36:48 @monitor.py:361] ranking_loss: 0.052676
[1028 21:36:48 @monitor.py:361] ranking_mean_average_precision: 0.75509
[1028 21:36:48 @monitor.py:361] training_auc: 0.90354
[1028 21:36:48 @group.py:42] Callbac

100%|##########|55/55[01:57<00:00, 0.45it/s]

[1028 21:38:46 @base.py:267] Epoch 20 (global_step 1100) finished, time:117.81 sec.



100%|##########|12/12[00:10<00:00, 1.20it/s]


[1028 21:38:56 @saver.py:89] Model saved to train_log/transfer/all_stages-g2/model-1100.
[1028 21:38:56 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 49.767
[1028 21:38:56 @monitor.py:361] QueueInput/queue_size_1: 49.686
[1028 21:38:56 @monitor.py:361] coverage: 4.2109
[1028 21:38:56 @monitor.py:361] learning_rate: 1e-05
[1028 21:38:56 @monitor.py:361] loss/value: 0.034595
[1028 21:38:56 @monitor.py:361] macro_auc: 0.93664
[1028 21:38:56 @monitor.py:361] macro_f1: 0.4186
[1028 21:38:56 @monitor.py:361] mean_average_precision: 0.66618
[1028 21:38:56 @monitor.py:361] micro_auc: 0.94765
[1028 21:38:56 @monitor.py:361] micro_f1: 0.54706
[1028 21:38:56 @monitor.py:361] one_error: 0.30078
[1028 21:38:56 @monitor.py:361] ranking_loss: 0.055729
[1028 21:38:56 @monitor.py:361] ranking_mean_average_precision: 0.74979
[1028 21:38:56 @monitor.py:361] training_auc: 0.90712
[1028 21:38:56 @group.py:42] Callbacks took 10.279 sec in total. DataParallelInferenceRunner: 10.057sec

In [15]:
test_loc_dir = 'train_log/transfer/test/D1'
test_save_name = 'max-training-auc.tfmodel'

config.proportion = {'train': 0.8, 'val': 0.2, 'test': 0.0}
config.stages = [2]
config.annotation_number = 10
dm = DataManager.from_dataset(train_set, test_set, config)
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
test_date = dm.get_test_stream()
model = RNN(config, is_finetuning=True)

{'posterior midgut primordium', 'embryonic brain', 'embryonic midgut', 'embryonic hindgut', 'trunk mesoderm primordium', 'faint ubiquitous', 'ventral nerve cord', 'anterior midgut primordium', 'ubiquitous', 'cellular blastoderm'}
Group numbers:
train: 580, validation: 128, test: 0
Image numbers:
train: 2241, validation: 345, test: 0
Group numbers:
train: 0, validation: 0, test: 637
Image numbers:
train: 0, validation: 0, test: 2278


/home/fuxiaofeng/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:160: RuntimeWarning: divide by zero encountered in true_divide
  return (1 - posi_ratio) / posi_ratio


In [18]:
dm.get_imbalance_ratio()

/home/fuxiaofeng/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:160: RuntimeWarning: divide by zero encountered in true_divide
  return (1 - posi_ratio) / posi_ratio


,train,val,test
posterior midgut primordium,inf,inf,inf
embryonic brain,inf,inf,inf
embryonic midgut,inf,inf,inf
embryonic hindgut,inf,inf,inf
trunk mesoderm primordium,inf,inf,inf
faint ubiquitous,4.576923,4.565217,3.976562
ventral nerve cord,inf,inf,inf
anterior midgut primordium,inf,inf,inf
ubiquitous,0.563342,0.422222,0.513064
cellular blastoderm,0.710914,0.542169,0.676316


In [22]:
train_set

,,image_url,annotation
gene,stage,,
CG32656,6,"(32951_s.bmp, 32954_s.bmp, 32952_s.bmp)","(imaginal tracheal precursor, embryonic head e..."
CG3028,5,"(38372_s.bmp, 38371_s.bmp, 38374_s.bmp)","(ubiquitous, ring gland primordium)"
CG4091,6,"(9653_s.bmp, 9654_s.bmp)","(imaginal tracheal precursor, adult hindgut pr..."
CG4608,5,"(15290_s.bmp, 15298_s.bmp, 15299_s.bmp, 15289_...","(dorsal epidermis primordium, tracheal primord..."
CG3722,6,"(18010_s.bmp, 18015_s.bmp, 18020_s.bmp, 18011_...","(embryonic head epidermis, embryonic foregut, ..."
CG9184,5,"(25526_s.bmp, 25527_s.bmp, 25529_s.bmp, 25530_...","(dorsal epidermis primordium, dorsomedial neur..."
CG12653,2,"(75726_s.bmp, 75727_s.bmp, 75728_s.bmp, 75729_...","(cellular blastoderm, procephalic ectoderm anl..."
CG4123,5,"(23368_s.bmp, 23368_s.bmp, 23364_s.bmp, 23365_...","(dorsal epidermis primordium, tracheal primord..."
CG7055,5,"(17943_s.bmp, 17944_s.bmp, 17945_s.bmp, 17941_...","(embryonic central brain glia, lateral cord gl..."


In [ ]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', test_save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=log_dir + save_name, ignore=ignore_restore),
                           max_epoch=20, tower=[0, 1])
Trainer(train_config).train()